In [4]:
from pymongo import MongoClient
import time
import pymongo
from pprint import pprint

# Step 1: Connect to MongoDB - Note: Change connection string as needed

client = MongoClient(port=27017)
db = client.admin
db.authors.drop_indexes()
db.comments.drop_indexes()

In [5]:
print('Total Number of cursors: ', len(list(db.cursors.distinct("cursor_id"))))
print('Total Number of comments: ', len(list(db.comments.distinct("comment_id"))))
print('Total Number of users: ', len(list(db.authors.distinct("author_id"))))

Total Number of cursors:  1739
Total Number of comments:  40258
Total Number of users:  7940


In [3]:
print('Total Number of cursors: ', len(list(db.cursors.find({}))))
print('Total Number of comments: ', len(list(db.comments.find({}))))
print('Total Number of users: ', len(list(db.authors.find({}))))

Total Number of cursors:  2891
Total Number of comments:  526618
Total Number of users:  7940


In [6]:
# Query 1
pprint(db.authors.find({'location': ''}).explain()['executionStats'])
pprint(db.command('serverStatus')['mem'])

output = db.authors.find({'location': ''})
print('Users who do not give their locations:')

for obj in output[:10]:
    print(obj['author_id'])

{'allPlansExecution': [],
 'executionStages': {'advanced': 6927,
                     'direction': 'forward',
                     'docsExamined': 7940,
                     'executionTimeMillisEstimate': 13,
                     'filter': {'location': {'$eq': ''}},
                     'isEOF': 1,
                     'nReturned': 6927,
                     'needTime': 1014,
                     'needYield': 0,
                     'restoreState': 7,
                     'saveState': 7,
                     'stage': 'COLLSCAN',
                     'works': 7942},
 'executionSuccess': True,
 'executionTimeMillis': 36,
 'nReturned': 6927,
 'totalDocsExamined': 7940,
 'totalKeysExamined': 0}
{'bits': 64, 'resident': 26, 'supported': True, 'virtual': 5809}
Users who do not give their locations:
1863371
322900440
108081345
1535699
291463672
289655103
50408726
151220172
4724491
106496403


In [18]:
# Query 2
start = time.time()
output = db.authors.aggregate([

    {"$match": {"location": {"$ne": ""}}},
    {"$group": {"_id": "$location", "count": {"$sum": 1}, "author_id": {"$addToSet": "$author_id"}}},
    {"$match": {"count": {"$gt": 1}}},
    {"$sort": {"count": -1}}],

)

end = time.time()

print('Query time is: ', end - start)

pprint(db.command('serverStatus')['mem'])

print('Users with the same location: ')
for result_object in output:
    print(result_object)

Query time is:  0.021323204040527344
{'bits': 64, 'resident': 62, 'supported': True, 'virtual': 6652}
Users with the same location: 
{'_id': 'USA', 'count': 39, 'author_id': ['400492', '213630211', '288321091', '144383316', '158648456', '163310003', '244363355', '183186500', '121767204', '122790872', '121737480', '79791732', '251499572', '104685427', '358638', '18889851', '88135433', '166350617', '49744963', '28013433', '182942535', '200331110', '46857031', '257498234', '212658600', '191959666', '324851491', '70494310', '50051531', '285766716', '155321424', '2694982', '106215523', '286998891', '96448623', '240210167', '7273692', '42332779', '9185673']}
{'_id': 'Texas', 'count': 23, 'author_id': ['164609587', '41664277', '52403083', '290837375', '108949760', '99619658', '1322910', '160234885', '3055712', '210881417', '62303106', '98583587', '164325964', '244939192', '111764185', '3964939', '108075579', '114076874', '119451312', '267200776', '211560240', '285111863', '2832674']}
{'_id': 

In [24]:
# Query 3
pprint(db.comments.find({
    '$and': [
        {'$or': [{"message": {'$regex': 'assault'}}, {"message": {'$regex': 'violence'}}]},
        {"message": {'$regex': 'bad'}}
    ]
}).explain()['executionStats'])

pprint(db.command('serverStatus')['mem'])

output = db.comments.distinct("author_id", {
    '$and': [
        {'$or': [{"message": {'$regex': 'assault'}}, {"message": {'$regex': 'violence'}}]},
        {"message": {'$regex': 'bad'}}
    ]
})

print('Users use same offensive words in their posts: ')
for result_object in output:
     if result_object != 'Unknown':
        pprint(result_object)

{'allPlansExecution': [],
 'executionStages': {'advanced': 52,
                     'direction': 'forward',
                     'docsExamined': 526618,
                     'executionTimeMillisEstimate': 72,
                     'filter': {'$and': [{'$or': [{'message': {'$regex': 'assault'}},
                                                  {'message': {'$regex': 'violence'}}]},
                                         {'message': {'$regex': 'bad'}}]},
                     'isEOF': 1,
                     'nReturned': 52,
                     'needTime': 526567,
                     'needYield': 0,
                     'restoreState': 526,
                     'saveState': 526,
                     'stage': 'COLLSCAN',
                     'works': 526620},
 'executionSuccess': True,
 'executionTimeMillis': 821,
 'nReturned': 52,
 'totalDocsExamined': 526618,
 'totalKeysExamined': 0}
{'bits': 64, 'resident': 838, 'supported': True, 'virtual': 6688}
Users use same offensive words in t

In [25]:
# Query 4
start = time.time()
output = db.authors.aggregate([

    {"$match": {"$and":
                [{"author_id": {"$in": db.comments.distinct("author_id", {"message": {"$regex": "bad"}})}},
                 {"location": {"$ne": ""}}]}},

    {"$group": {"_id": "$location", "count": {"$sum": 1}, "author_id": {"$addToSet": "$author_id"}}},
    {"$match": {"count": {"$gt": 1}}},
    {"$sort": {"count": -1}},

])
end = time.time()

print('Query time is: ', end - start)

pprint(db.command('serverStatus')['mem'])

print('Users with the same location and same word: ')
for result_object in output:
    pprint(result_object)

Query time is:  0.466900110244751
{'bits': 64, 'resident': 840, 'supported': True, 'virtual': 6688}
Users with the same location and same word: 
{'_id': 'Earth',
 'author_id': ['245851695', '232208082', '288432140'],
 'count': 3}
{'_id': 'USA', 'author_id': ['49744963', '50051531'], 'count': 2}


In [21]:
# Query 5
start = time.time()
output = db.comments.aggregate([
    {"$group": {"_id": "$author_id", "count": {"$sum": 1}}},
    {"$match": {"$and": [{"count": {"$gt": 5}}, {"author_id": {"$ne": ""}}]}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
])
end = time.time()
print('Query execution time: ', end - start)

pprint(db.command('serverStatus')['mem'])

print('Top-k users by their number of messages: ')
for result_object in output:
    if result_object['_id'] != 'Unknown':
        pprint(result_object)

Query execution time:  0.9522128105163574
{'bits': 64, 'resident': 462, 'supported': True, 'virtual': 6649}
Top-k users by their number of messages: 
{'_id': '288536631', 'count': 1543}
{'_id': '292842000', 'count': 1400}
{'_id': '118923992', 'count': 1201}
{'_id': '294944884', 'count': 1143}
{'_id': '150140509', 'count': 1136}
{'_id': '280239772', 'count': 840}
{'_id': '324921053', 'count': 814}
{'_id': '233610691', 'count': 773}
{'_id': '322920136', 'count': 716}
{'_id': '78916407', 'count': 696}


In [15]:
# Query 6
start = time.time()
output = db.authors.aggregate([
    {"$match": {"author_id": {"$nin": db.comments.distinct("author_id", {"parent_comment_id": None})}}},
    {"$project": {"author_id": 1, "comments.parent_comment_id": 1}}
])
end = time.time()
print('Query execution time: ', end - start)

pprint(db.command('serverStatus')['mem'])

print('Users who only reply to others: ')

for result_object in output:
    pprint(result_object)

Query execution time:  0.5667562484741211
{'bits': 64, 'resident': 506, 'supported': True, 'virtual': 6685}
Users who only reply to others: 
{'_id': ObjectId('5fcd551c6f2553f57631cdff'), 'author_id': '108081345'}
{'_id': ObjectId('5fcd551c6f2553f57631ce2d'), 'author_id': '4724491'}
{'_id': ObjectId('5fcd551d6f2553f57631ce80'), 'author_id': '21799234'}
{'_id': ObjectId('5fcd551d6f2553f57631ced0'), 'author_id': '23497889'}
{'_id': ObjectId('5fcd551d6f2553f57631cf17'), 'author_id': '9863759'}
{'_id': ObjectId('5fcd551d6f2553f57631cf50'), 'author_id': '38221864'}
{'_id': ObjectId('5fcd551d6f2553f57631cf56'), 'author_id': '114083166'}
{'_id': ObjectId('5fcd551d6f2553f57631cf5e'), 'author_id': '25655101'}
{'_id': ObjectId('5fcd551d6f2553f57631cf61'), 'author_id': '110786496'}
{'_id': ObjectId('5fcd551d6f2553f57631cf91'), 'author_id': '8333448'}
{'_id': ObjectId('5fcd551d6f2553f57631cffc'), 'author_id': '323854030'}
{'_id': ObjectId('5fcd551d6f2553f57631d092'), 'author_id': '154246411'}
{'_id

In [16]:
# Query 7
start = time.time()
output = db.authors.aggregate([
    {"$match": {"author_id": {"$nin": db.comments.distinct("author_id", {"parent_comment_id": {"$ne": None}})}}},
    {"$project": {"author_id": 1}}
])
end = time.time()
print('Query execution time: ', end - start)

pprint(db.command('serverStatus')['mem'])

print('Users who never reply to other messages: ')
for result_object in output:
    pprint(result_object)

Query execution time:  0.6492607593536377
{'bits': 64, 'resident': 505, 'supported': True, 'virtual': 6682}
Users who never reply to other messages: 
{'_id': ObjectId('5fcd551d6f2553f57631cf2d'), 'author_id': '39660942'}
{'_id': ObjectId('5fcd551d6f2553f57631cf3e'), 'author_id': '2878180'}
{'_id': ObjectId('5fcd551d6f2553f57631cf41'), 'author_id': '261050468'}
{'_id': ObjectId('5fcd551d6f2553f57631cf47'), 'author_id': '23887464'}
{'_id': ObjectId('5fcd551d6f2553f57631cf4a'), 'author_id': '358495'}
{'_id': ObjectId('5fcd551d6f2553f57631cf53'), 'author_id': '284317071'}
{'_id': ObjectId('5fcd551d6f2553f57631cf59'), 'author_id': '17345871'}
{'_id': ObjectId('5fcd551d6f2553f57631cf6b'), 'author_id': '28726205'}
{'_id': ObjectId('5fcd551d6f2553f57631cf73'), 'author_id': '155675061'}
{'_id': ObjectId('5fcd551d6f2553f57631cf83'), 'author_id': '5465516'}
{'_id': ObjectId('5fcd551d6f2553f57631cfb2'), 'author_id': '76575566'}
{'_id': ObjectId('5fcd551d6f2553f57631cfd1'), 'author_id': '292844595'

{'_id': ObjectId('5fcd55966f2553f57633310d'), 'author_id': '168894139'}
{'_id': ObjectId('5fcd55966f2553f57633315f'), 'author_id': '193153629'}
{'_id': ObjectId('5fcd55966f2553f576333162'), 'author_id': '7081230'}
{'_id': ObjectId('5fcd55966f2553f576333180'), 'author_id': '152213102'}
{'_id': ObjectId('5fcd55966f2553f576333189'), 'author_id': '281225120'}
{'_id': ObjectId('5fcd55966f2553f57633319a'), 'author_id': '255192092'}
{'_id': ObjectId('5fcd55966f2553f5763331a5'), 'author_id': '250865881'}
{'_id': ObjectId('5fcd55966f2553f5763331ba'), 'author_id': '324724650'}
{'_id': ObjectId('5fcd55966f2553f5763331c1'), 'author_id': '215523959'}
{'_id': ObjectId('5fcd55966f2553f5763331cc'), 'author_id': '230530588'}
{'_id': ObjectId('5fcd55966f2553f5763331cf'), 'author_id': '3339402'}
{'_id': ObjectId('5fcd55966f2553f5763331dd'), 'author_id': '24047440'}
{'_id': ObjectId('5fcd55966f2553f5763331ee'), 'author_id': '297480518'}
{'_id': ObjectId('5fcd55966f2553f5763331f1'), 'author_id': '92003546'

{'_id': ObjectId('5fcd56306f2553f576344002'), 'author_id': '260354915'}
{'_id': ObjectId('5fcd56306f2553f57634400f'), 'author_id': '239060367'}
{'_id': ObjectId('5fcd56306f2553f576344048'), 'author_id': '174317280'}
{'_id': ObjectId('5fcd56306f2553f57634404b'), 'author_id': '196955896'}
{'_id': ObjectId('5fcd56306f2553f5763440d9'), 'author_id': '323006077'}
{'_id': ObjectId('5fcd56306f2553f576344124'), 'author_id': '252562018'}
{'_id': ObjectId('5fcd56316f2553f57634415e'), 'author_id': '103160'}
{'_id': ObjectId('5fcd56316f2553f576344173'), 'author_id': '199960180'}
{'_id': ObjectId('5fcd56316f2553f576344185'), 'author_id': '166208551'}
{'_id': ObjectId('5fcd56316f2553f5763441a0'), 'author_id': '30008336'}
{'_id': ObjectId('5fcd56686f2553f576348aad'), 'author_id': '57516830'}
{'_id': ObjectId('5fcd56736f2553f5763499d3'), 'author_id': '324674509'}
{'_id': ObjectId('5fcd56996f2553f57634cc9a'), 'author_id': '155562868'}
{'_id': ObjectId('5fcd56c66f2553f576350891'), 'author_id': '23820845'

In [17]:
# Query 8
start = time.time()
output = db.comments.aggregate([
    {"$group":
         {"_id": "$author_id",
          "avgLikes": {"$avg": "$comment_likes"},
          "avgDislikes": {"$avg": "$comment_dislikes"}}},
    {"$match": {"avgLikes": {"$gt": 1}}},
    {"$sort": {"avgLikes": -1}}
])
end = time.time()
print('Query execution time: ', end - start)

pprint(db.command('serverStatus')['mem'])


print('Avg likes and dislikes per user: ')
for result_object in output:
    pprint(result_object)

Query execution time:  1.0705020427703857
{'bits': 64, 'resident': 506, 'supported': True, 'virtual': 6682}
Avg likes and dislikes per user: 
{'_id': '17452698', 'avgDislikes': 0.0, 'avgLikes': 725.5}
{'_id': '131413377', 'avgDislikes': 0.0, 'avgLikes': 396.125}
{'_id': '110434834', 'avgDislikes': 0.0, 'avgLikes': 229.16666666666666}
{'_id': '283927074', 'avgDislikes': 1.0, 'avgLikes': 197.28571428571428}
{'_id': '284417180', 'avgDislikes': 0.0, 'avgLikes': 192.0}
{'_id': '243945682', 'avgDislikes': 0.0, 'avgLikes': 184.0}
{'_id': '276155630',
 'avgDislikes': 0.11764705882352941,
 'avgLikes': 140.7450980392157}
{'_id': '219318371', 'avgDislikes': 0.0, 'avgLikes': 136.13793103448276}
{'_id': '255680944', 'avgDislikes': 0.0, 'avgLikes': 126.42857142857143}
{'_id': '156852461', 'avgDislikes': 0.0, 'avgLikes': 110.25}
{'_id': '164990184', 'avgDislikes': 6.0, 'avgLikes': 106.875}
{'_id': '2979643', 'avgDislikes': 0.0, 'avgLikes': 102.66666666666667}
{'_id': '258228092', 'avgDislikes': 0.0, 

{'_id': '109336918',
 'avgDislikes': 0.2857142857142857,
 'avgLikes': 2.857142857142857}
{'_id': '35842288', 'avgDislikes': 0.0, 'avgLikes': 2.857142857142857}
{'_id': '239688284', 'avgDislikes': 0.0, 'avgLikes': 2.857142857142857}
{'_id': '168261241', 'avgDislikes': 0.0, 'avgLikes': 2.857142857142857}
{'_id': '66726564', 'avgDislikes': 0.0, 'avgLikes': 2.857142857142857}
{'_id': '132863206', 'avgDislikes': 0.0, 'avgLikes': 2.857142857142857}
{'_id': '230242148', 'avgDislikes': 0.0, 'avgLikes': 2.857142857142857}
{'_id': '120636918', 'avgDislikes': 0.0, 'avgLikes': 2.857142857142857}
{'_id': '125222788',
 'avgDislikes': 0.14285714285714285,
 'avgLikes': 2.857142857142857}
{'_id': '217404555',
 'avgDislikes': 0.010736196319018405,
 'avgLikes': 2.854294478527607}
{'_id': '283343062', 'avgDislikes': 0.0, 'avgLikes': 2.85}
{'_id': '255834081',
 'avgDislikes': 0.13793103448275862,
 'avgLikes': 2.8448275862068964}
{'_id': '246134448', 'avgDislikes': 0.0, 'avgLikes': 2.8440366972477062}
{'_id

{'_id': '55170465', 'avgDislikes': 0.0, 'avgLikes': 1.9230769230769231}
{'_id': '35647499', 'avgDislikes': 0.0, 'avgLikes': 1.9230769230769231}
{'_id': '182099884', 'avgDislikes': 0.0, 'avgLikes': 1.9230769230769231}
{'_id': '277784004', 'avgDislikes': 0.0, 'avgLikes': 1.919463087248322}
{'_id': '228635816', 'avgDislikes': 0.0, 'avgLikes': 1.9166666666666667}
{'_id': '157489724', 'avgDislikes': 0.0, 'avgLikes': 1.9166666666666667}
{'_id': '190793875', 'avgDislikes': 0.0, 'avgLikes': 1.9166666666666667}
{'_id': '66122143', 'avgDislikes': 0.0, 'avgLikes': 1.9166666666666667}
{'_id': '167630560', 'avgDislikes': 0.0, 'avgLikes': 1.9166666666666667}
{'_id': '67542065', 'avgDislikes': 0.0, 'avgLikes': 1.9152542372881356}
{'_id': '93038910', 'avgDislikes': 0.0, 'avgLikes': 1.911764705882353}
{'_id': '25013571', 'avgDislikes': 0.0, 'avgLikes': 1.9090909090909092}
{'_id': '7874834', 'avgDislikes': 0.0, 'avgLikes': 1.9}
{'_id': '199367998', 'avgDislikes': 0.0, 'avgLikes': 1.9}
{'_id': '151994452

{'_id': '141471188', 'avgDislikes': 0.0, 'avgLikes': 1.1923076923076923}
{'_id': '267387949', 'avgDislikes': 0.0, 'avgLikes': 1.1919191919191918}
{'_id': '116556507', 'avgDislikes': 0.0, 'avgLikes': 1.1914893617021276}
{'_id': '75973272', 'avgDislikes': 0.0, 'avgLikes': 1.1911764705882353}
{'_id': '324720322', 'avgDislikes': 0.0, 'avgLikes': 1.1904761904761905}
{'_id': '292715519',
 'avgDislikes': 0.09523809523809523,
 'avgLikes': 1.1904761904761905}
{'_id': '296210369', 'avgDislikes': 0.0, 'avgLikes': 1.1891891891891893}
{'_id': '182819297',
 'avgDislikes': 0.05660377358490566,
 'avgLikes': 1.1886792452830188}
{'_id': '45125600', 'avgDislikes': 0.0, 'avgLikes': 1.1886792452830188}
{'_id': '267432040', 'avgDislikes': 0.0, 'avgLikes': 1.1875}
{'_id': '269553726',
 'avgDislikes': 0.25477707006369427,
 'avgLikes': 1.186836518046709}
{'_id': '37262038', 'avgDislikes': 0.0, 'avgLikes': 1.186046511627907}
{'_id': '244334568',
 'avgDislikes': 0.24074074074074073,
 'avgLikes': 1.18518518518518

In [7]:
# Batch execution
num = 5
start = time.time()
for _ in range(num):
    db.authors.find({'location': ''})
for _ in range(num):
    db.authors.aggregate([

        {"$match": {"location": {"$ne": ""}}},
        {"$group": {"_id": "$location", "count": {"$sum": 1}, "author_id": {"$addToSet": "$author_id"}}},
        {"$match": {"count": {"$gt": 5}}},
        {"$sort": {"count": -1}}],
    )
for _ in range(num):
    db.comments.distinct("author_id", {
        '$and': [
            {'$or': [{"message": {'$regex': 'assault'}}, {"message": {'$regex': 'violence'}}]},
            {"message": {'$regex': 'bad'}}
        ]
    })
for _ in range(num):
    db.authors.aggregate([

        {"$match": {"$and":
                    [{"author_id": {"$in": db.comments.distinct("author_id", {"message": {"$regex": "bad"}})}},
                     {"location": {"$ne": ""}}]}},

        {"$group": {"_id": "$location", "count": {"$sum": 1}, "author_id": {"$addToSet": "$author_id"}}},
        {"$match": {"count": {"$gt": 1}}},
        {"$sort": {"count": -1}},

    ])

for _ in range(num):

    db.comments.aggregate([
        {"$group": {"_id": "$author_id", "count": {"$sum": 1}}},
        {"$match": {"$and": [{"count": {"$gt": 5}}, {"author_id": {"$ne": ""}}]}},
        {"$sort": {"count": -1}},
        {"$limit": 10}
    ])
for _ in range(num):
    db.authors.aggregate([
        {"$match": {"author_id": {"$nin": db.comments.distinct("author_id", {"parent_comment_id": None})}}},
        {"$project": {"author_id": 1, "comments.parent_comment_id": 1}}
    ])

for _ in range(num):
    db.authors.aggregate([
        {"$match": {"author_id": {"$nin": db.comments.distinct("author_id", {"parent_comment_id": {"$ne": None}})}}},
        {"$project": {"author_id": 1}}
    ])

for _ in range(num):
    db.comments.aggregate([
        {"$group":
             {"_id": "$author_id",
              "avgLikes": {"$avg": "$comment_likes"},
              "avgDislikes": {"$avg": "$comment_dislikes"}}},
        {"$match": {"avgLikes": {"$gt": 5}}},
        {"$sort": {"avgLikes": -1}}
    ])
end = time.time()
print('Query time: ', end - start)

Query time:  84.816330909729
